<a href="https://colab.research.google.com/github/Areej17-01/Generative-AI-LLMs-and-Transformers./blob/main/T5_for_product_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Feb  4 07:13:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install numpy
!pip install transformers[torch]
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

In [4]:
dataset_category = "Software"

meta_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_meta_{dataset_category}", split='full').to_pandas()[['parent_asin', 'title']]
review_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_review_{dataset_category}", split='full').to_pandas()[['parent_asin', 'rating', 'text', 'verified_purchase']]

ds = meta_ds.merge(review_ds, on='parent_asin', how='inner').drop(columns="parent_asin")
ds = ds.rename(columns={"rating":"star_rating", "title":"product_title", "text":"review_body"})

ds = ds[ds['verified_purchase'] & (ds['review_body'].map(len) > 100)].sample(100_000)
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

The repository for McAuley-Lab/Amazon-Reviews-2023 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/McAuley-Lab/Amazon-Reviews-2023.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


meta_Software.jsonl:   0%|          | 0.00/256M [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

Software.jsonl:   0%|          | 0.00/1.87G [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

,product_title,star_rating,review_body,verified_purchase
1855021,"WeatherBug - Free Local Weather Forecast, Rada...",5.0,I love it! It's great when I have to figure o...,True
1892699,World Of Hidden Objects - a 100% object findin...,5.0,Love this game. As have others my eyes are not...,True
4044913,Pic IQ Quiz: Reveal the Pic,4.0,This is a fun game. Not overly challenging whi...,True
4598176,Spider,5.0,I love playing card games. I have this app on ...,True
3084084,Bugs War,3.0,"I hate bugs and wanted to try this, all I have...",True
...,...,...,...,...
342770,Buttons and Scissors (Pro),5.0,Really makes you think about what you're doing...,True
1049926,AccuWeather Platinum with Superior Accuracy™,3.0,"lots of detail, but have to admit sometimes I ...",True
2342190,Video For YouTube,3.0,The thing about this you tube app is I can't g...,True
4181191,Open One Photo II,5.0,I really. like this game very fun and makes yo...,True


In [8]:
ds=ds[:50000]

In [9]:
# Loading the dataset
dataset = Dataset.from_pandas(ds)

# encoding the 'star_rating' column
dataset = dataset.class_encode_column("star_rating")

# Splitting the dataset into training and testing sets
dataset = dataset.train_test_split(test_size=0.1, seed=42, stratify_by_column="star_rating")

train_dataset = dataset['train']
test_dataset = dataset['test']
print(train_dataset[0])

Stringifying the column:   0%|          | 0/50000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'product_title': 'The Wizard of Oz Magic Match 3', 'star_rating': 0, 'review_body': 'Emailed support since I play EVERYDAY since this game started and it did not give me my 21 day bonus, when I was at day 18 it took me back to day 1! Pissed after spending money.', 'verified_purchase': True, '__index_level_0__': 2826041}


In [10]:
train_dataset

Dataset({
    features: ['product_title', 'star_rating', 'review_body', 'verified_purchase', '__index_level_0__'],
    num_rows: 45000
})

In [11]:
MODEL_NAME = 't5-base'
tokenizer = T5Tokenizer.from_pretrained('t5-base')

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
# Defining the function to preprocess the data
def preprocess_data(examples):
    examples['prompt'] = [f"review: {product_title}, {star_rating} Stars!" for product_title, star_rating in zip(examples['product_title'], examples['star_rating'])]
    examples['response'] = [f"{review_body}" for review_body in examples['review_body']]

    inputs = tokenizer(examples['prompt'], padding='max_length', truncation=True, max_length=128)
    targets = tokenizer(examples['response'], padding='max_length', truncation=True, max_length=128)

    # Set -100 at the padding positions of target tokens
    target_input_ids = []
    for ids in targets['input_ids']:
        target_input_ids.append([id if id != tokenizer.pad_token_id else -100 for id in ids])

    inputs.update({'labels': target_input_ids})
    return inputs

In [13]:
train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [38]:
test_dataset

Dataset({
    features: ['product_title', 'star_rating', 'review_body', 'verified_purchase', '__index_level_0__', 'prompt', 'response', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [16]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

TRAINING_OUTPUT = "./models/t5_fine_tuned_reviews"
training_args = TrainingArguments(
    output_dir=TRAINING_OUTPUT,
    num_train_epochs=3,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    save_strategy='epoch',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mehboobareej01 (mehboobareej01-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,3.576000
1000,3.440700
1500,3.390900
2000,3.354500
2500,3.332300
3000,3.323900
3500,3.307300
4000,3.287300
4500,3.263200
5000,3.254700


TrainOutput(global_step=11250, training_loss=3.2784602159288196, metrics={'train_runtime': 8819.1489, 'train_samples_per_second': 15.308, 'train_steps_per_second': 1.276, 'total_flos': 2.05523288064e+16, 'train_loss': 3.2784602159288196, 'epoch': 3.0})

In [18]:

from huggingface_hub import login
from google.colab import userdata
hf_login=userdata.get('hf_login')
login(hf_login)

In [20]:
from huggingface_hub import create_repo

# Create a repository
repo_name = "T5-base-amazon-reviews"
create_repo(repo_id=repo_name, private=True)  # Set private=True if you want it to be private


RepoUrl('https://huggingface.co/Areeej/T5-base-amazon-reviews', endpoint='https://huggingface.co', repo_type='model', repo_id='Areeej/T5-base-amazon-reviews')

In [31]:
model_dir = "/content/models/t5_fine_tuned_reviews/checkpoint-11250"
repo_id = "Areeej/T5-base-amazon-reviews"  # Correct format for the repo_id

# Now upload the folder to the Hugging Face Hub
upload_folder(
    folder_path=model_dir,
    repo_id=repo_id,  # Correct repo_id format
)


CommitInfo(commit_url='https://huggingface.co/Areeej/T5-base-amazon-reviews/commit/fefb53a4dac9a2765e94cce6faa74e8c6a29416a', commit_message='Upload folder using huggingface_hub', commit_description='', oid='fefb53a4dac9a2765e94cce6faa74e8c6a29416a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Areeej/T5-base-amazon-reviews', endpoint='https://huggingface.co', repo_type='model', repo_id='Areeej/T5-base-amazon-reviews'), pr_revision=None, pr_num=None)

In [25]:
from transformers import AutoConfig

# Replace "gpt2" with your base model type if it's different
config = AutoConfig.from_pretrained("t5-base")

# Customize if needed
config.save_pretrained("t5-base")



In [33]:
model = T5ForConditionalGeneration.from_pretrained("Areeej/T5-base-amazon-reviews")


config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [34]:
def generate_review(text):
    inputs = tokenizer("review: " + text, return_tensors='pt', max_length=512, padding='max_length', truncation=True)
    outputs = model.generate(inputs['input_ids'], max_length=128, no_repeat_ngram_size=3, num_beams=6, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [35]:
random_products = test_dataset.shuffle(42).select(range(10))['product_title']

print(generate_review(random_products[0] + ", 3 Stars!"))
print(generate_review(random_products[1] + ", 5 Stars!"))
print(generate_review(random_products[2] + ", 2 Stars!"))

This is a fun game to play. The only thing I don't like is that you have to wait for a certain amount of time to play the game.
This is a great crossword puzzle game. It's easy to play and easy to understand. I would recommend this game to anyone who likes puzzles.
It's a fun game, but it takes a while to get the hang of it. I haven't played it in a long time.


In [37]:
print(generate_review(random_products[2] + ", 0 Stars!"))

I've been playing this game for about a year now and it's a waste of my time. It's not worth the money.
